Exploración inicial del dataset LANL

En este primer notebook se va a realizar una exploración preeliminar de dataset de autenticación de Los Alamos National Laboratory (LANL).
El objetivo de este primer notebook es comprender la estructura y validar su idoneidad para la detección de anomalías en entornos IAM.

In [1]:
import pandas as pd
import numpy as np

A continuación, vamos a definir el nombre de las cabeceras para diferenciar los tipos de datos que hay en la BBDD.
Para poder distinguir cada una de las columnas, se ha seguido la información proporcionada por la página web donde se encuentra el dataset.

In [2]:
#Definición del nombre de las cabeceras
columns = [
    "time",
    "src_user",
    "dst_user",
    "src_comp",
    "dst_comp",
    "auth_type",
    "logon_type",
    "auth_orientation",
    "auth_result"
]

Se realiza una primera carga parcial del dataset.

In [3]:
#Se cargan un total de 100000 filas iniciales para poder distinguir el tipo de información que aparece.
df = pd.read_csv(
    "auth.txt",
    names=columns,
    nrows=500000
)

df.head()

,time,src_user,dst_user,src_comp,dst_comp,auth_type,logon_type,auth_orientation,auth_result
0,1,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C1250,C586,NTLM,Network,LogOn,Success
1,1,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success
2,1,C101$@DOM1,C101$@DOM1,C988,C988,?,Network,LogOff,Success
3,1,C1020$@DOM1,SYSTEM@C1020,C1020,C1020,Negotiate,Service,LogOn,Success
4,1,C1021$@DOM1,C1021$@DOM1,C1021,C625,Kerberos,Network,LogOn,Success


In [4]:
df

,time,src_user,dst_user,src_comp,dst_comp,auth_type,logon_type,auth_orientation,auth_result
0,1,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C1250,C586,NTLM,Network,LogOn,Success
1,1,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success
2,1,C101$@DOM1,C101$@DOM1,C988,C988,?,Network,LogOff,Success
3,1,C1020$@DOM1,SYSTEM@C1020,C1020,C1020,Negotiate,Service,LogOn,Success
4,1,C1021$@DOM1,C1021$@DOM1,C1021,C625,Kerberos,Network,LogOn,Success
...,...,...,...,...,...,...,...,...,...
499995,5118,U292@DOM1,C1737$@DOM1,C1737,C1737,?,?,AuthMap,Success
499996,5118,U292@DOM1,U292@DOM1,C105,C104,?,?,TGS,Success
499997,5118,U292@DOM1,U292@DOM1,C105,C105,?,?,TGT,Success
499998,5118,U292@DOM1,U292@DOM1,C1737,C1737,?,?,TGS,Success


Ahora que se ha cargado parcialmente los datos del dataset y conocemos las columnas en las que se divide, se realizará un analisis del comportamiento de los datos.

DIMENSIÓN Y VOLUMEN DE DATOS

In [ ]:
#Comprobación del numero de filas, columnas, tipos de datos en cada columna, la cantidad de valores nulos y el uso de memoria total.
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype
---  ------            --------------   -----
 0   time              500000 non-null  int64
 1   src_user          500000 non-null  str  
 2   dst_user          500000 non-null  str  
 3   src_comp          500000 non-null  str  
 4   dst_comp          500000 non-null  str  
 5   auth_type         500000 non-null  str  
 6   logon_type        500000 non-null  str  
 7   auth_orientation  500000 non-null  str  
 8   auth_result       500000 non-null  str  
dtypes: int64(1), str(8)
memory usage: 34.3 MB


In [6]:
df.shape

(500000, 9)

ANÁLISIS ESTADÍSTICO BÁSICO

In [7]:
df.describe(include="all")

,time,src_user,dst_user,src_comp,dst_comp,auth_type,logon_type,auth_orientation,auth_result
count,500000.000000,500000,500000,500000,500000,500000,500000,500000,500000
unique,NaN,6574,8215,4062,3824,11,9,5,2
top,NaN,U22@DOM1,U22@DOM1,C586,C586,?,Network,LogOff,Success
freq,NaN,13750,13750,32313,66270,290874,414579,215845,496592
mean,2541.068868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1492.077579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1232.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2545.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,3852.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


ANÁLISIS TEMPORAL

In [ ]:
df['auth_result'].value_counts()


auth_result
Success    496592
Fail         3408
Name: count, dtype: int64

In [9]:
3408./496592

0.006862776685891033

In [10]:
df["src_user"].nunique()

6574

CONVERSIÓN TEMPORAL

In [11]:
df["time"] = pd.to_datetime(df["time"], unit="s")
df["time"].head()

0   1970-01-01 00:00:01
1   1970-01-01 00:00:01
2   1970-01-01 00:00:01
3   1970-01-01 00:00:01
4   1970-01-01 00:00:01
Name: time, dtype: datetime64[s]